# Read in all the files, and compile them into on big .csv file

### I will use Simbad names, where available, to match everything together

*Andrew Bowen provided some of the script to read in the files. [See his GitHub repo](https://github.com/andrewbowen19/CEB_Project)*

Some alterations to files:
- changed Pismis 6 to NGC 2645 in MWSC, Piskunov and Kharchenko
- Luginbuhl-Skiff_1 is Skiff_J0614+12.9 (?) changed in WEBDA
- Skiff_2 is Skiff_J0458+43.0 (?) changed in WEBDA
- PTB 9 is a planetary nebula (not an OC --though maybe in the OC NGC 7762?) -- changed label in MWSC, removed from Kharchenko
- Collinder_258 is Harvard_5 (same PM, distance, etc.), both are in Gaia table, removed Harvard_5
- FSR_1686 is Juchert_10, changed to Juchert 10 in MWSC and Kharchenko
- changed many BH or VBH, etc. to vdBergh-Hagen
- removed Berkeley 42 from vandenbergh and Salaris because it's a GC (and included in Harris)
- removed Mrk 38 from vandenbergh because it's a pair of galaxies
- removed Mrk 50 from vandenbergh because it's a Seyfert 1 galaxy
- vdBergh-Hagen_133 is Collinder_258 is Harvard 5,removed vdBergh-Hagen 133 in lynga
- Berkeley_30 is Biurakan_9, changed to Berkeley 30 in lynga
- Berkeley_32 Biurakan_8, changed to Berkeley 32 in lynga
- removed NGC 281 from lunga (HII region)
- not sure about NGC_2579 and AH03_J0822-36.4 (possible overlap)
- removed vdBergh-Hagen_1 from vandenberg = reflection nebula
- Cl VDBH 47 is IC 2395, remove vdBergh-Hagen 47 from WEBDA and lynga
- vdBergh-Hagen_218 might be NGC_6318 , removed vdBergh-Hagen_218 from lynga
- NAME_HD_80077_Group is Pismis_11 and vdBergh-Hagen_60, removed vdBergh-Hagen_60 from lynga
- from Simbad: "NGC 2239 is an obsolete designation for NGC 2244.", removed NGC 2239 from lynga
- NAME_Trapezium_Cluster M_42, should I change Trapezium in lynga to M42? I think yes



In [1]:
import pandas as pd
import numpy as np
import sys
import time
import re

from astropy.coordinates import SkyCoord
from astropy import units 

from astroquery.simbad import Simbad
Simbad.TIMEOUT = 300 # sets the timeout to 60s

import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)



In [2]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [3]:
def matchToSimbad(cat_ID, fillCat = True):
    #I'd like to use the Simbad names for matching (if possible) so I reduce the duplicates, 

    match_ID = np.array([str(x) for x in cat_ID], dtype='object')
    return_ID = np.full(len(match_ID), np.nan, dtype='object')

    #fix the names
    #is this correct? : .replace('vdBergh','CL VDBH')
    for i, x in enumerate(match_ID):
        match_ID[i] = x.replace('vdBergh-Hagen_','Cl_VDBH_')\
                        .replace('vdBergh_','Cl_VDB_')\
                        .replace('FSR_','[FSR2007]_')\
                        .replace('DBSB_','[DBS2003]_')\
                        .replace('BDSB_','[BDS2003]_')\
                        .replace('BDS_','[BDS2003]_')\
                        .replace('Alessi_','Cl_Alessi_')\
                        .replace('Bica_','Cl_Bica_')\
                        .replace('Sher_','Cl_Sher_')\
                        .replace('MWSC_','[KPS2012]_MWSC_')\
                        .replace('Schuster_1','NAME_SCHUSTER_CL')\
                        .replace('ASCC_','[KPR2005]_')\
                        .replace('Andrews-Lindsay_', '[AL67]_Cl*')\
                        .replace('Arp-Madore', 'AM')\
                        .replace('LDN_988e', 'NAME_[C86]_L988_e_Cluster')\
                        .replace('Juchert-Saloran_','Juchert-Saloranta_')\
                        .replace('Ivanov', '[IBP2002]_CC')\
                        .replace('DB2001_','[DB2001]_Cl ')\
                        .replace('Havlen-Moffat_','Cl_HM ')\
                        .replace('Cl_Cl','Cl')\
                        .replace('[KPS2012]_[KPS2012]','[KPS2012]')\

#     result_table = Simbad.query_objects(match_ID) #stupidly, this does not return blank rows for missing data!
# below I tried matching by RA and Dec, but I'm finding that (according to authors) there are quite a few 
# unique clusters, that are very close together, and it's not clear how to get avoid mismatches.
# If it returned blank lines, then this wouldn't be an issue!!
# So I'm trying to go one line at a time, but of course Simbad doesn't like that... it rejects my connection when
# I have too many requests (and what is too many?  and how long should I wait?)
    for i, ID in enumerate(match_ID):
        result_table = Simbad.query_object(ID)
        if (result_table):
            #print(result_table)
            if (len(result_table) > 0):
                row = result_table[0]
                newID =  re.sub('\s+', ' ', row['MAIN_ID'].decode("utf-8")).strip().replace(' ','_' )
                return_ID[i] = newID
        print(f'{i} {float(i)/len(match_ID):5.3f}, {ID} {return_ID[i]} -- ', end='')
        if (i>0 and i % 50 == 0):
            time.sleep(30)
            print('\n\n')
        time.sleep(0.1)

    #fill nans with the original catalog search input?
    if (fillCat):
        for i, ID in enumerate(return_ID):
            if (pd.isna(ID)):
                return_ID[i] = match_ID[i]
        
    return return_ID

In [4]:
fixMWSC = False
fixWEBDA = False
fixPiskunov = False
fixKharchenko = False
fixSalaris = False
fixVandenBergh = False
fixGaia = False
fixLynga = False

In [5]:
#MWSC - Milky Way Star Clusters Catalog
#https://heasarc.gsfc.nasa.gov/W3Browse/all/mwsc.html
if (fixMWSC):
    mwsc_df = pd.read_csv("MWSC.txt", header=3, delimiter='|')
    mwsc_df.columns = mwsc_df.columns.str.strip()

    
    #take only the open clusters
    mwsc_df = mwsc_df.loc[(~mwsc_df['class'].str.strip().str.contains('GLOBULAR')) &
                          (~mwsc_df['class'].str.strip().str.contains('NEBULA')) &
                          (~mwsc_df['class'].str.strip().str.contains('UNIDENTIFIED'))]

    #remove the "Unnamed" columns
    mwsc_df = mwsc_df.loc[:, ~mwsc_df.columns.str.contains('^Unnamed')]

    print(set(mwsc_df['class'].values))
    
    #fix the IDs
    mwsc_ID = mwsc_df['Name']
    mwsc_ID = mwsc_ID.str.strip().str.replace( ' ','_' )
    mwsc_df['Name'] = mwsc_ID

    #add Simbad names for matching (if possible) so I reduce the duplicates 
    ID = [x for x in mwsc_df['Name']]
    simbad_ID = matchToSimbad(mwsc_ID)
    
    mwsc_df.columns = [str(col) + '_mwsc' for col in mwsc_df.columns]
    mwsc_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)

    #check for duplicates
    for index, row in  mwsc_df.iterrows():
        check = mwsc_df.loc[mwsc_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('MWSC', row['Simbad_ID'], row['Name_mwsc'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
        
    mwsc_df.to_csv('MWSC_clean.csv', index=False)
    

mwsc_df = pd.read_csv('MWSC_clean.csv')
mwsc_df

,Simbad_ID,Name_mwsc,broad_type_mwsc,cluster_status_mwsc,ra_mwsc,dec_mwsc,lii_mwsc,bii_mwsc,core_radius_mwsc,central_radius_mwsc,cluster_radius_mwsc,pm_ra_mwsc,pm_dec_mwsc,pm_tot_error_mwsc,rad_vel_mwsc,rad_vel_error_mwsc,num_rad_vel_stars_mwsc,num_core_stars_mwsc,num_central_stars_mwsc,num_cluster_stars_mwsc,distance_mwsc,e_bv_mwsc,distance_modulus_mwsc,e_jk_mwsc,e_jh_mwsc,delta_h_mwsc,log_age_mwsc,log_age_error_mwsc,num_log_age_stars_mwsc,king_core_radius_mwsc,king_core_radius_error_mwsc,king_tidal_radius_mwsc,king_tidal_radius_error_mwsc,king_norm_factor_mwsc,king_norm_factor_error_mwsc,reference_code_mwsc,cluster_type_mwsc,metallicity_mwsc,metallicity_error_mwsc,num_metallicity_stars_mwsc,comments_mwsc,class_mwsc
0,[KPS2012]_MWSC_4688,MWSC_4688,,,23 51 54,-86 43.2,303.907,-30.295,0.020,0.100,0.185,3.20,-5.00,1.13,,,,2,22,57,1336,0.219,10.700,0.105,0.070,0.000,9.390,,,1.05,0.39,7.01,2.51,2.51,0.67,AIPk,,,,0,...,OPEN STAR CL...
1,[KPS2012]_MWSC_5684,MWSC_5684,,,12 53 43,-86 38.9,302.968,-23.776,0.020,0.080,0.155,-13.04,0.17,1.05,,,,3,19,52,1432,0.375,10.900,0.180,0.120,0.020,9.180,0.023,3,0.61,0.42,7.54,5.79,1.88,1.09,ARIs,,,,0,...,OPEN STAR CL...
2,[KPS2012]_MWSC_5692,MWSC_5692,,,17 47 20,-86 36.6,306.562,-26.146,0.025,0.095,0.135,-6.22,-9.02,1.38,,,,4,18,28,1555,0.437,11.100,0.210,0.140,0.020,8.930,,,0.76,0.56,5.18,3.82,1.51,0.77,ARIs,,,,0,...,OPEN STAR CL...
3,[KPS2012]_MWSC_4005,MWSC_4005,,,00 11 28,-85 28.8,303.852,-31.577,0.012,0.100,0.165,9.31,-1.47,0.95,,,,3,26,42,1159,0.250,10.400,0.120,0.080,-0.020,9.375,,,0.36,0.13,4.69,1.85,15.94,6.19,AIPk,,,,0,...,OPEN STAR CL...
4,[KPS2012]_MWSC_4176,MWSC_4176,,,14 27 18,-85 25.2,304.950,-22.929,0.025,0.150,0.280,-9.41,0.12,0.69,,,,4,53,155,1093,0.333,10.300,0.160,0.107,-0.030,9.315,,,0.97,0.24,6.84,1.58,6.46,1.31,AIPk,,,,0,...,OPEN STAR CL...
5,ESO_8-6,ESO_008-06,r,c,14 56 55,-83 26.7,306.593,-21.485,0.025,0.130,0.185,-5.26,-4.21,0.80,,,,5,46,78,1380,0.312,10.800,0.150,0.100,0.030,9.300,,,0.66,0.25,5.60,2.18,5.93,1.91,DIAS,,,,0,"Sparse; center is shifted to 14.9485h,-83.445d...",OPEN STAR CL...
6,[KPS2012]_MWSC_4219,MWSC_4219,,,15 42 14,-83 11.7,307.905,-22.041,0.015,0.115,0.200,-9.26,-3.14,0.74,,,,2,36,95,1606,0.375,11.150,0.180,0.120,0.020,9.100,,,2.81,0.76,9.78,2.13,1.71,0.41,AIPk,,,,0,...,OPEN STAR CL...
7,[KPS2012]_MWSC_5575,MWSC_5575,,,01 59 42,-83 03.0,300.484,-33.751,0.015,0.090,0.150,6.33,-2.50,1.96,,,,1,10,20,2191,0.302,11.800,0.145,0.097,0.015,9.200,,,1.95,1.00,12.13,6.13,0.79,0.18,ARIs,,,,0,Poor RDP. ...,OPEN STAR CL...
8,[KPS2012]_MWSC_4682,MWSC_4682,,,23 43 23,-82 57.6,305.407,-33.838,0.020,0.115,0.190,5.87,0.28,0.94,,,,2,22,58,1065,0.354,10.250,0.170,0.113,0.000,9.280,0.061,4,0.43,0.24,8.27,5.39,5.33,3.02,AIPk,,,,0,...,OPEN STAR CL...
9,[KPS2012]_MWSC_5685,MWSC_5685,,,13 05 24,-82 02.6,303.443,-19.185,0.020,0.090,0.160,-3.12,-2.73,1.14,,,,3,25,64,1581,0.406,11.125,0.195,0.130,0.000,9.150,,,0.65,0.21,7.54,2.68,9.02,2.40,ARIs,,,,0,Poor RDP. ...,OPEN STAR CL...


### Downloaded from WEBDA [here](https://webda.physics.muni.cz/cluster_selall.html)

With RA from 0 to 24 and 0 to 1e6 stars. I copied the table to WEBDA.html, and removed the $<$br$>$ entries, then converted to csv with 

https://codepen.io/malahovks/pen/gLxLWX

or

https://jsfiddle.net/gengns/j1jm2tjx/

Finally, I separated the RA and DEC column header into 2 entries.

*I also have a data file from David James, that has more clusters, but I'm not sure the providence of that, so I won't use it.*

In [6]:
# # WEBDA data file (2013)
# webda_df = pd.read_fwf("WEBDA-OC-table-June2013_DavidJames.txt", 
#                        widths = [18,14,15,11,9,8,8,8,9,6,9,9,9,7,7,9], header = 0)

if (fixWEBDA):
    webda_df = pd.read_csv('WEBDA.csv')
    #fix the IDs
    webda_ID = webda_df['Name']
    webda_ID = webda_ID.str.replace( 'NGC 0','NGC ' ).str.replace( ' ','_' )

    webda_df['Name'] = webda_ID

    #add Simbad names for matching (if possible) so I reduce the duplicates 
    simbad_ID = matchToSimbad(webda_ID)

    webda_df.columns = [str(col) + '_webda' for col in webda_df.columns]
    webda_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)
        
    #check for duplicates  
    for index, row in  webda_df.iterrows():
        check = webda_df.loc[webda_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('WEBDA', row['Simbad_ID'], row['Name_webda'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
        
    webda_df.to_csv('WEBDA_clean.csv', index=False)
    
webda_df = pd.read_csv('WEBDA_clean.csv')
webda_df

,Simbad_ID,Name_webda,RA_2000_webda,Dec_2000_webda,l_webda,b_webda,Dist_webda,Mod_webda,EB-V_webda,Age_webda,ST_webda,Z_webda,Diam_webda,Fe/H_webda,MRV_webda,pm RA_webda,pm Dec_webda,Measures_webda,Stars_webda
0,Cl_Berkeley_58,Berkeley_58,00 00 12,+60 58 00,116.753,-1.289,3715.0,14.55,0.550,8.400,NaN,-83.6,5.0,NaN,NaN,NaN,NaN,525,519
1,Cl_Stock_18,Stock_18,00 01 37,+64 37 30,117.624,2.268,2800.0,14.41,0.700,6.780,B0,110.8,6.0,NaN,NaN,NaN,NaN,2261,2261
2,Cl_Berkeley_59,Berkeley_59,00 02 13,+67 25 11,118.220,5.000,1000.0,13.78,1.220,6.800,NaN,87.2,20.4,NaN,-6.50,-4.40,0.73,27,21
3,Cl_Blanco_1,Blanco_1,00 04 07,-29 50 00,15.572,-79.261,269.0,7.18,0.010,7.796,B5,-264.3,70.0,0.23,NaN,20.17,3.00,109,105
4,[KPR2005]_1,ASCC_1,00 09 35,+62 40 48,118.150,0.190,4000.0,13.51,0.160,8.250,NaN,13.3,24.0,NaN,-76.15,-2.07,0.46,32,32
5,Cl_Berkeley_1,Berkeley_1,00 09 36,+60 28 30,117.796,-1.979,2420.0,14.35,0.780,8.600,NaN,-83.6,5.0,NaN,NaN,NaN,NaN,2800,2800
6,C_0007+609,King_13,00 10 06,+61 10 00,117.968,-1.306,3100.0,15.00,0.820,8.500,NaN,-70.7,5.0,NaN,NaN,NaN,NaN,4253,3955
7,[KPR2004b]_4,Alessi_20,00 10 33,+58 45 35,117.640,-3.690,450.0,8.95,0.220,8.220,NaN,-29.0,36.0,NaN,NaN,7.48,-2.61,42,42
8,[KPR2005]_2,ASCC_2,00 19 51,+55 42 35,118.460,-6.890,1200.0,10.71,0.100,8.830,NaN,-144.0,36.0,NaN,NaN,-0.91,-3.94,57,57
9,NAME_Cl_Mayer_1,Mayer_1,00 21 54,+61 44 24,119.440,-0.930,1429.0,12.02,0.400,7.740,NaN,-23.2,24.0,NaN,-20.90,-5.27,-5.87,15146,15131


In [7]:
#Piskunov (2008)
if (fixPiskunov):
    piskunov_df = pd.read_fwf("Piskunov2008.table", 
                              widths = [6,18,7,7,8,6,6,6,6,6,9,9,6,6,9,9], header = None,
                              names = ['COCD','Name','GLON[deg]','GLAT[deg]','DistMod','E(B-V)','Dist[pc]',\
                                       'logt[yr]','rt[pc]','e_rt[pc]','logM[MSun]','e_logM[MSun]','rtA[pc]','e_rtA[pc]',
                                       'logMA[MSun]','e_logMA[MSun]'])

    piskunov_df.replace(-9.999,np.nan, inplace=True)
    piskunov_df.replace(-9.9,np.nan, inplace=True)

    #fix the IDs
    piskunov_ID = piskunov_df['Name']
    piskunov_ID = piskunov_ID.str.strip().str.replace(' ','_' )
    piskunov_df['Name'] = piskunov_ID
    
    #add Simbad names for matching (if possible) so I reduce the duplicates 
    simbad_ID = matchToSimbad(piskunov_ID)
    
    piskunov_df.columns = [str(col) + '_piskunov' for col in piskunov_df.columns]
    piskunov_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)

    #check for duplicates
    for index, row in piskunov_df.iterrows():
        check = piskunov_df.loc[piskunov_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('Piskunov', row['Simbad_ID'], row['Name_piskunov'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
        
    piskunov_df.to_csv('Piskunov2008_clean.csv', index=False)

piskunov_df = pd.read_csv('Piskunov2008_clean.csv')
piskunov_df

,Simbad_ID,COCD_piskunov,Name_piskunov,GLON[deg]_piskunov,GLAT[deg]_piskunov,DistMod_piskunov,E(B-V)_piskunov,Dist[pc]_piskunov,logt[yr]_piskunov,rt[pc]_piskunov,e_rt[pc]_piskunov,logM[MSun]_piskunov,e_logM[MSun]_piskunov,rtA[pc]_piskunov,e_rtA[pc]_piskunov,logMA[MSun]_piskunov,e_logMA[MSun]_piskunov
0,Cl_Berkeley_58,1,Berkeley_58,116.73,-1.29,14.555,0.55,3715,8.20,NaN,NaN,NaN,NaN,22.9,10.9,3.380,0.623
1,Cl_Berkeley_59,2,Berkeley_59,118.22,5.00,13.782,1.22,1000,6.80,NaN,NaN,NaN,NaN,8.0,3.5,2.221,0.564
2,Cl_Blanco_1,3,Blanco_1,14.17,-79.02,7.180,0.01,269,8.32,22.8,3.8,3.646,0.219,20.0,2.4,3.480,0.160
3,[KPR2004b]_4,4,Alessi_20,117.64,-3.69,8.948,0.22,450,8.22,5.4,1.6,1.742,0.391,4.0,0.8,1.362,0.250
4,NAME_Cl_Mayer_1,5,Mayer_1,119.44,-0.93,12.015,0.40,1429,7.74,NaN,NaN,NaN,NaN,16.7,5.6,3.150,0.442
5,Cl_Stock_20,6,Stock_20,119.92,-0.10,10.413,0.20,909,8.53,NaN,NaN,NaN,NaN,6.6,1.3,1.974,0.260
6,C_0027+577,7,Stock_21,120.05,-4.83,11.447,0.40,1100,8.72,NaN,NaN,NaN,NaN,8.3,2.4,2.259,0.376
7,NGC_129,8,NGC_129,120.27,-2.54,12.759,0.55,1625,7.87,14.9,2.7,2.984,0.240,15.2,2.7,3.011,0.235
8,NGC_146,9,NGC_146,120.87,0.50,13.897,0.48,3032,7.37,NaN,NaN,NaN,NaN,20.8,7.1,3.298,0.445
9,NGC_225,10,NGC_225,122.01,-1.08,9.925,0.27,657,8.19,NaN,NaN,NaN,NaN,5.7,1.3,1.787,0.311


In [8]:
#Kharchenko (2013)
if (fixKharchenko):
    kharchenko_df = pd.read_fwf("Kharchenko2013.table", 
                              widths = [5,18,2,1,9,8,8,8,7,7,7,7,7,7,8,8,6,6,7,7,8,7,7,7,7,7,7,7,4,8,8,8,8,8,8,5,4,8,7,4], 
                              header = None,
                              names = ['MWSC','Name','Type','n_Type','RA[hr]','Dec[deg]','GLON[deg]','GLAT[deg]',
                                       'r0[deg]','r1[deg]','r2[deg]','pmRA[mas/yr]','pmDec[mas/yr]','e_pm[mas/yr]',
                                       'RV[km/s]','e_RV[km/s]','n_RV[km/s]','N1sr0','N1sr1','N1sr2','d[pc]','E(B-V)',
                                       'appDistMod[mag]','E(J-Ks)','E(J-H)','dH','logt[yr]','e_logt[yr]','Nt','rc[pc]',
                                       'e_rc[pc]','rt[pc]','e_rt[pc]','k[pc-2]','e_k[pc-2]','Src','SType','[Fe/H][Sun]',
                                       'e_[Fe/H][Sun]','n_[Fe/H]'])

    kharchenko_df['RV[km/s]'].replace(999.99,np.nan, inplace=True)
    kharchenko_df['e_RV[km/s]'].replace(99.99,np.nan, inplace=True)
    kharchenko_df['e_logt[yr]'].replace(0.000,np.nan, inplace=True)
    kharchenko_df['Nt'].replace(-1,np.nan, inplace=True)
    kharchenko_df['rc[pc]'].replace(0.00,np.nan, inplace=True)
    kharchenko_df['e_rc[pc]'].replace(0.00,np.nan, inplace=True)
    kharchenko_df['rt[pc]'].replace(0.00,np.nan, inplace=True)
    kharchenko_df['e_rt[pc]'].replace(0.00,np.nan, inplace=True)
    kharchenko_df['k[pc-2]'].replace(0.00,np.nan, inplace=True)
    kharchenko_df['e_k[pc-2]'].replace(0.00,np.nan, inplace=True)
    kharchenko_df['[Fe/H][Sun]'].replace(99.999,np.nan, inplace=True)
    kharchenko_df['e_[Fe/H][Sun]'].replace(9.99,np.nan, inplace=True)
    kharchenko_df['n_[Fe/H]'].replace(0.1,np.nan, inplace=True)

    #kharchenko_df.loc[kharchenko_df['Name'] == 'Skiff_J0458+43.0']
    
    #add Simbad names for matching (if possible) so I reduce the duplicates 
    kharchenko_ID = kharchenko_df['Name']
    simbad_ID = matchToSimbad(kharchenko_ID)

    kharchenko_df.columns = [str(col) + '_kharchenko' for col in kharchenko_df.columns]
    kharchenko_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)
    
    #check for duplicates
    for index, row in kharchenko_df.iterrows():
        check = kharchenko_df.loc[kharchenko_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('Kharchenko', row['Simbad_ID'], row['Name_kharchenko'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
            
    kharchenko_df.to_csv('Kharchenko2013_clean.csv', index=False)

kharchenko_df = pd.read_csv('Kharchenko2013_clean.csv')
kharchenko_df

,Simbad_ID,MWSC_kharchenko,Name_kharchenko,Type_kharchenko,n_Type_kharchenko,RA[hr]_kharchenko,Dec[deg]_kharchenko,GLON[deg]_kharchenko,GLAT[deg]_kharchenko,r0[deg]_kharchenko,r1[deg]_kharchenko,r2[deg]_kharchenko,pmRA[mas/yr]_kharchenko,pmDec[mas/yr]_kharchenko,e_pm[mas/yr]_kharchenko,RV[km/s]_kharchenko,e_RV[km/s]_kharchenko,n_RV[km/s]_kharchenko,N1sr0_kharchenko,N1sr1_kharchenko,N1sr2_kharchenko,d[pc]_kharchenko,E(B-V)_kharchenko,appDistMod[mag]_kharchenko,E(J-Ks)_kharchenko,E(J-H)_kharchenko,dH_kharchenko,logt[yr]_kharchenko,e_logt[yr]_kharchenko,Nt_kharchenko,rc[pc]_kharchenko,e_rc[pc]_kharchenko,rt[pc]_kharchenko,e_rt[pc]_kharchenko,k[pc-2]_kharchenko,e_k[pc-2]_kharchenko,Src_kharchenko,SType_kharchenko,[Fe/H][Sun]_kharchenko,e_[Fe/H][Sun]_kharchenko,n_[Fe/H]_kharchenko
0,Cl_Berkeley_58,1,Berkeley_58,NaN,NaN,0.0045,60.933,116.750,-1.326,0.025,0.087,0.155,0.56,1.56,0.26,NaN,NaN,0,16,88,197,2700.0,0.720,12.389,0.346,0.231,0.000,8.470,0.047,10.0,1.12,0.25,13.66,3.31,5.19,1.00,COCD,NaN,NaN,NaN,0
1,NGC_7801,2,NGC_7801,NaN,NaN,0.0055,50.727,114.717,-11.331,0.015,0.070,0.156,-3.20,-3.47,0.71,NaN,NaN,0,2,14,65,1953.0,0.146,11.500,0.070,0.047,0.000,9.255,NaN,1.0,0.61,0.33,9.93,6.14,2.67,1.44,DIAS,NaN,NaN,NaN,0
2,[KPS2012]_MWSC_0003,3,FSR_0459,NaN,NaN,0.0085,59.242,116.446,-2.990,0.018,0.055,0.090,-1.66,-0.01,0.53,NaN,NaN,0,3,24,50,2926.0,1.145,12.700,0.550,0.367,0.000,7.800,NaN,NaN,0.39,0.21,7.65,4.63,5.62,3.53,DIAS,irc,NaN,NaN,0
3,Cl_Stock_18,4,Stock_18,NaN,NaN,0.0265,64.625,117.617,2.266,0.010,0.050,0.080,-3.59,-1.15,0.54,NaN,NaN,0,2,20,32,774.0,0.177,9.501,0.085,0.057,-0.030,8.680,NaN,1.0,0.16,0.03,2.14,0.38,355.57,56.43,DIAS,NaN,NaN,NaN,0
4,Cl_Berkeley_59,5,Berkeley_59,NaN,NaN,0.0373,67.425,118.219,5.001,0.035,0.115,0.220,-3.20,-1.11,0.38,-12.50,7.08,3,16,45,89,1000.0,1.241,10.399,0.596,0.398,-0.040,6.100,NaN,NaN,0.55,0.06,6.51,0.74,153.96,12.16,COCD,NaN,NaN,NaN,0
5,Ass_Cep_OB_4,6,Cep_OB4,ao,NaN,0.0490,67.500,118.299,5.062,0.120,0.345,0.760,-0.93,-2.58,0.21,0.00,NaN,1,61,198,654,850.0,1.099,10.001,0.528,0.352,-0.030,6.100,NaN,NaN,9.54,2.64,13.56,1.39,16.73,10.54,MELN,ass,NaN,NaN,0
6,Cl_Blanco_1,7,Blanco_1,NaN,NaN,0.0590,-30.000,14.830,-79.098,0.500,1.400,2.350,19.71,2.28,0.18,5.48,2.04,24,82,214,266,250.0,0.012,6.994,0.006,0.004,-0.030,7.750,NaN,NaN,2.82,0.38,10.93,1.26,24.84,2.43,COCD,NaN,-0.188,0.098,7
7,Cl_Berkeley_104,8,Berkeley_104,NaN,NaN,0.0568,63.580,117.615,1.202,0.018,0.055,0.135,-4.57,-4.37,0.44,NaN,NaN,0,8,34,136,3599.0,0.606,12.976,0.291,0.194,-0.005,8.805,0.031,15.0,1.13,0.27,14.92,3.92,4.51,0.99,DIAS,NaN,0.070,NaN,0
8,IRAS_00013+6817,9,IRAS_00013+6817,NaN,NaN,0.0664,68.565,118.595,6.091,0.010,0.055,0.090,-3.52,-1.55,0.87,NaN,NaN,0,2,9,23,1338.0,0.833,10.900,0.400,0.267,-0.020,7.150,NaN,NaN,0.73,0.33,5.69,2.54,1.69,0.61,BIEM,irc,NaN,NaN,0
9,[KPS2012]_MWSC_0014,14,FSR_0504,NaN,NaN,0.0970,81.840,121.227,19.116,0.010,0.045,0.080,-3.57,7.83,0.68,NaN,NaN,0,2,10,14,3744.0,0.104,12.900,0.050,0.033,-0.020,9.450,NaN,NaN,0.49,0.19,10.62,4.26,5.03,2.46,FPOS,NaN,NaN,NaN,0


In [9]:
# Salaris (2004) -- no RA or Dec...
if (fixSalaris):
    #salaris_df = pd.read_csv('Salaris2004_viaWEBDA.txt', sep='\t', header = 15)
    salaris_df = pd.read_csv('Salaris2004_table1.txt', sep='\t')
    #print(salaris_df)

    #change the Hyades to Melotte 25 so that it matches with van den Bergh for position
    names = salaris_df['Name'].values
    xx = np.where(names == 'Hyades')
    names[xx] = 'Melotte_25'
    salaris_df['Name'] = [x.strip().replace(' ','_' ) for x in names]
    xx = np.where(names == 'Arp-Madore_2')
    names[xx] = 'AM_2'  

    simbad_ID = matchToSimbad(names)

    salaris_df.columns = [str(col) + '_salaris' for col in salaris_df.columns]
    salaris_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)
    
    #check for duplicates
    for index, row in salaris_df.iterrows():
        check = salaris_df.loc[salaris_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('Salaris', row['Simbad_ID'], row['Name_salaris'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
            
    salaris_df.to_csv('Salaris2004_table1_clean.csv', index=False)

salaris_df = pd.read_csv('Salaris2004_table1_clean.csv')
salaris_df

,Simbad_ID,Name_salaris,dV_salaris,err_dV_salaris,[Fe/H]_salaris,err_[Fe/H]_salaris,t[Gyr]_salaris,err_t_salaris,Rgc[kpc]_salaris,z[pc]_salaris,flag_salaris,tJP94[Gyr]_salaris
0,Cl_King_2,King_2,2.2,0.15,0.00,0.20,5.03,1.31,12.98,-510,2,5.6
1,IC_166,IC_166,1.0,0.25,-0.27,0.15,1.32,0.43,10.74,-10,1,1.5
2,NGC_752,NGC_752,0.9,0.05,-0.09,0.06,1.24,0.20,8.75,-145,1,1.4
3,Cl_Berkeley_66,Berkeley_66,2.0,0.25,0.00,0.20,3.98,1.52,12.59,20,2,4.4
4,NGC_1193,NGC_1193,2.1,0.15,-0.35,0.11,4.23,1.08,12.00,-845,1,4.9
5,C_0311+525,King_5,0.4,0.15,-0.30,0.15,0.76,0.16,10.34,-163,2,0.9
6,NGC_1245,NGC_1245,0.7,0.15,0.10,0.15,1.06,0.23,11.09,-465,1,1.0
7,NGC_1798,NGC_1798,1.0,0.15,-0.47,0.15,1.28,0.29,11.79,290,2,1.5
8,NGC_1817,NGC_1817,0.8,0.05,-0.10,0.09,1.12,0.18,10.26,-410,1,1.3
9,Cl_Berkeley_17,Berkeley_17,2.8,0.15,-0.33,0.15,10.06,2.77,10.89,-155,1,12.6


In [10]:
#van den Bergh (2006)
#there were two rows for Berkeley 69, with slightly different values.  I kept the first one.
if (fixVandenBergh):
    vandenbergh_df = pd.read_csv('vandenbergh2006.tsv', sep='|', header = 49)

    #fix the names
    names = vandenbergh_df['SimbadName'].values
    def representsInt(s):
        try: 
            int(s)
            return True
        except ValueError:
            return False

    for i,x in enumerate(names):
        if (x[0:2] == 'Cl'):
            names[i] = x[2:]
        if (x[0:1] == 'N' and representsInt(x[1:1])):
            names[i] = 'NGC '+x[1:]
        names[i] = names[i].strip().replace('  ',' ').replace( ' ','_' )

    vandenbergh_df['Name'] = names
    
    #add Simbad names for matching (if possible) so I reduce the duplicates 
    vandenbergh_ID = vandenbergh_df['Name']
    simbad_ID = matchToSimbad(vandenbergh_ID)

    vandenbergh_df.columns = [str(col) + '_vandenbergh' for col in vandenbergh_df.columns]
    vandenbergh_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)
    
    #check for duplicates
    for index, row in vandenbergh_df.iterrows():
        check = vandenbergh_df.loc[vandenbergh_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('van den Bergh', row['Simbad_ID'], row['Name_vandenbergh'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
        
        
    vandenbergh_df.to_csv('vandenbergh2006_clean.csv', index=False)

vandenbergh_df = pd.read_csv('vandenbergh2006_clean.csv')
vandenbergh_df

,Simbad_ID,Seq_vandenbergh,Name_vandenbergh,l[deg]_vandenbergh,Diam[pc]_vandenbergh,R[pc]_vandenbergh,Z[pc]_vandenbergh,E(B-V)_vandenbergh,logT[yr]_vandenbergh,SimbadName_vandenbergh,_RA[deg]_vandenbergh,_Dec[deg]_vandenbergh
0,C_1756-281,1,Trumpler_31,2,1.43,986,-39,0.35,8.87,Trumpler_31,269.97500,-28.18333
1,NGC_6520,2,NGC_6520,2,2.29,1577,-78,0.43,7.72,NGC_6520,270.85000,-27.88333
2,NGC_6530,3,NGC_6530,6,5.42,1330,-31,0.33,6.87,NGC_6530,271.12500,-24.36667
3,Cl_Bochum_14,4,Bochum_14,6,0.34,578,-5,1.51,7.00,Bochum_14,270.50000,-23.70000
4,M_20,5,NGC_6514,7,6.65,816,-4,0.19,7.37,NGC_6514,270.59583,-23.03000
5,NGC_6546,6,NGC_6546,7,3.82,938,-23,0.49,7.85,NGC_6546,271.90000,-23.33333
6,M_21,7,NGC_6531,7,2.91,1205,-7,0.28,7.07,NGC_6531,271.05000,-22.48333
7,M_23,8,NGC_6494,9,5.30,628,31,0.36,8.48,NGC_6494,269.25000,-18.98333
8,Cl_Trumpler_33,10,Trumpler_33,12,2.55,1755,-99,0.36,7.68,Trumpler_33,276.17500,-19.71667
9,Cl_Collinder_469,11,Collinder_469,12,1.29,1481,-20,0.42,7.80,Collinder_469,274.10000,-18.21667


In [11]:
#Gaia DR2 from Cantat-Gaudin+, 2018
if (fixGaia):
    gaiaDR2_df = pd.read_csv('Cantat-Gaudin2018_GaiaDR2.tsv', sep='|', header = 56)

    names = gaiaDR2_df['Name']
    names = names.str.strip()

    for index, row in  gaiaDR2_df.iterrows():
    #for i,n in enumerate(names):
        if (names[index][0:3] == "ESO"):
            names[index] = row['SimbadName'].replace(' ','_')

    gaiaDR2_df['Name'] = names

    #this already contains the Simbad name, but they are not identical to what I'm finding 
    gaiaDR2_SID = gaiaDR2_df['SimbadName'].str.replace(' ','_').replace('Name','NAME').values
    idx = gaiaDR2_df.index[pd.isnull(gaiaDR2_df['SimbadName'])].tolist()
    gaiaDR2_SID[idx] = names[idx]
    simbad_ID = matchToSimbad(gaiaDR2_SID)
#     #because there was at least one bad match of Gulliver_15 to NGC 6561 (when Gulliver 15 is not in Simbad)
#     #also Gulliver 7 to Ruprecht 77
#     idx = gaiaDR2_df.index[pd.isnull(gaiaDR2_df['SimbadName'])].tolist()
#     simbad_ID[idx] = names[idx]


    gaiaDR2_df.columns = [str(col) + '_cantat-gaudin' for col in gaiaDR2_df.columns]
    gaiaDR2_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)

    #check for duplicates
    for index, row in  gaiaDR2_df.iterrows():
        check = gaiaDR2_df.loc[gaiaDR2_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('gaiaDR2', row['Simbad_ID'], row['Name_cantat-gaudin'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
        
    gaiaDR2_df.to_csv('Cantat-Gaudin2018_GaiaDR2_clean.csv', index=False)

gaiaDR2_df = pd.read_csv('Cantat-Gaudin2018_GaiaDR2_clean.csv')
gaiaDR2_df#.loc[(gaiaDR2_df['Name'] == 'Collinder_258') | (gaiaDR2_df['Name'] == 'Harvard_5')]

,Simbad_ID,_RAJ2000_cantat-gaudin,_DEJ2000_cantat-gaudin,Name_cantat-gaudin,RAJ2000[deg]_cantat-gaudin,DEJ2000[deg]_cantat-gaudin,GLON[deg]_cantat-gaudin,GLAT[deg]_cantat-gaudin,r50[deg]_cantat-gaudin,Nstars_cantat-gaudin,pmRA[mas/yr]_cantat-gaudin,pmDE[mas/yr]_cantat-gaudin,plx[mas]_cantat-gaudin,dmode[pc]_cantat-gaudin,Rgc[pc]_cantat-gaudin,SimbadName_cantat-gaudin
0,NAME_Alessi_Teutsch_9,03 27 28.80,+34 58 51.6,ASCC_10,51.870,34.981,155.723,-17.770,0.558,71,-1.737,-1.368,1.459,672.0,8927.2,[KPR2005] 10
1,[KPR2005]_101,19 13 35.76,+36 22 08.4,ASCC_101,288.399,36.369,68.028,11.608,0.372,75,0.934,1.288,2.488,397.3,8202.3,[KPR2005] 101
2,[KPR2005]_105,19 42 11.52,+27 21 57.6,ASCC_105,295.548,27.366,62.825,2.063,0.648,127,1.464,-1.635,1.783,551.8,8103.0,[KPR2005] 105
3,[KPR2005]_107,19 48 39.36,+21 59 13.2,ASCC_107,297.164,21.987,58.904,-1.901,0.174,59,-0.155,-5.156,1.109,878.5,7922.3,[KPR2005] 107
4,[KPR2005]_108,19 53 13.44,+39 20 56.4,ASCC_108,298.306,39.349,74.378,6.074,0.537,230,-0.519,-1.690,0.838,1154.0,8106.7,[KPR2005] 108
5,[KPR2005]_11,03 32 13.44,+44 51 21.6,ASCC_11,53.056,44.856,150.546,-9.224,0.312,276,0.926,-3.030,1.141,854.5,9083.9,[KPR2005] 11
6,[KPR2005]_110,20 02 58.08,+33 31 40.8,ASCC_110,300.742,33.528,70.411,1.378,0.203,70,0.271,-3.132,0.497,1902.2,7908.1,[KPR2005] 110
7,[KPR2005]_111,20 11 33.84,+37 30 54.0,ASCC_111,302.891,37.515,74.714,2.056,0.537,156,-1.150,-1.524,1.166,836.9,8159.5,[KPR2005] 111
8,[KPR2005]_113,21 11 43.92,+38 38 16.8,ASCC_113,317.933,38.638,82.877,-6.589,0.529,196,0.800,-3.679,1.762,558.2,8289.5,[KPR2005] 113
9,[KPR2005]_114,21 39 57.60,+53 59 49.2,ASCC_114,324.990,53.997,97.082,1.028,0.216,150,-3.716,-3.421,1.066,913.2,8501.0,[KPR2005] 114


In [30]:
#lynga catalo: https://heasarc.gsfc.nasa.gov/W3Browse/star-catalog/lyngaclust.html
if (fixLynga):
    lynga_df = pd.read_csv('lyngaCat.txt',sep='|')

    #fix the column names
    lynga_df.rename(columns=lambda x: x.strip(), inplace=True)

    #fix all the cells
    lynga_df = lynga_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    lynga_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

    #take only the first entries for each name (some have multiple lines
    lynga_df = lynga_df.loc[pd.notna(lynga_df['Name'].str.strip())]

    #fix capitalization in the names!
    names = lynga_df['Name'].values
    for i,n in enumerate(names):
        if ((n[0:3] != 'NGC') and (n[0:2] != 'IC')):
            p1 = n.find(' ')
            n = n[0] + n[1:p1].lower() + n[p1:]

        if (n[0:9] == 'Vdb-hagen'):
            n = 'vdBergh-Hagen' + n[9:]

        if (n[0:10] == 'Hav-moffat'):
            n = 'Havlen-Moffat' + n[10:]

        if (n[0:7] == 'Vdbergh'):
            n = 'vdBergh' + n[7:]

        if (n[0:8] == 'Dol-dzim'):
            n = 'Dol-Dzim' + n[8:]

        if (n == 'Sigma ORI'):
            n = 'Sigma Ori'

        if (n == 'Trapezium'):
            n = 'M 42'
            
        names[i] = n.replace(' ','_')

    lynga_df['Name'] = names

    #print(lynga_df.loc[lynga_df['Name'] == 'NGC_2579'])
    
    #remove the "Unnamed" columns
    lynga_df = lynga_df.loc[:, ~lynga_df.columns.str.contains('^Unnamed')]
    
    #add Simbad names for matching (if possible) so I reduce the duplicates 
    lynga_ID = lynga_df['Name']
    simbad_ID = matchToSimbad(lynga_ID)

    lynga_df.columns = [str(col) + '_lynga' for col in lynga_df.columns]
    lynga_df.insert(loc=0, column='Simbad_ID', value=simbad_ID)
    
    #check for duplicates
    for index, row in  lynga_df.iterrows():
        check = lynga_df.loc[lynga_df['Simbad_ID'] == row['Simbad_ID']]
        if (len(check['Simbad_ID']) != 1):
            print('lynga', row['Simbad_ID'], row['Name_lynga'])
            print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
            print('')
        
    lynga_df.to_csv('lyngaCat_clean.csv', index=False)
    
lynga_df = pd.read_csv('lyngaCat_clean.csv')
lynga_df

,Simbad_ID,Name_lynga,ra_lynga,dec_lynga,distance_lynga,log_age_lynga,angular_diameter_lynga,alt_name_lynga,lii_lynga,bii_lynga,iau_num_lynga,seq_code_lynga,seq_num_lynga,prec_ra_lynga,prec_dec_lynga,lund_record_num_lynga,ocl_num_lynga,ref_angular_diameter_lynga,ref_distance_lynga,ref_log_age_lynga,metallicity_lynga,ref_metallicity_lynga,e_bv_lynga,ref_e_bv_lynga,type_flag_lynga,ref_type_flag_lynga,tr_concent_class_lynga,tr_range_class_lynga,tr_richness_class_lynga,tr_nebulosity_lynga,sb_bs_mag_lynga,sb_spect_code_lynga,sb_total_mag_lynga,sk_total_mag_lynga,sk_bv_color_lynga,sk_num_stars_lynga,ja_star_num_lynga,ja_class_lynga,ja_max_class_lynga,ja_richness_lynga,ja_e_bv_lynga,ref_ja_e_bv_lynga,ja_bv_turnoff_lynga,ref_ja_bv_turnoff_lynga,ly_tr_concent_class_lynga,ly_tr_range_class_lynga,ly_tr_richness_class_lynga,ly_tr_nebulosity_lynga,ly_member_stars_lynga,ly_angular_diameter_lynga,ly_refs_flag_lynga,radvel_weight_lynga,radvel_lynga,radvel_weight_class_lynga,ref_radvel1_lynga,ref_radvel2_lynga,ref_radvel3_lynga,ref_radvel4_lynga,ref_radvel5_lynga,basel_spect_code_lynga,basel_color_type_lynga,neg_ra_tracer_lynga,neg_lii_tracer_lynga,neg_seq_num_tracer_lynga,pos_ra_tracer_lynga,pos_lii_tracer_lynga,pos_seq_num_tracer_lynga,jdl_distance_lynga,jdl_distance_weight_lynga,jdl_turnoff_color_lynga,jdl_age_lynga,jdl_age_weight_lynga,jdl_reddening_lynga,jdl_reddening_flag_lynga,jdl_reddening_weight_lynga,ref_jdl1_lynga,ref_jdl2_lynga,ref_jdl3_lynga,ref_jdl4_lynga,ref_jdl5_lynga,ref_jdl6_lynga
0,Cl_Berkeley_59,Berkeley_59,0.64425,67.37840,NaN,NaN,10.0,0000+671,118.25,4.95,C0000+671,3,59,0.52,3.3,3,286.0,419.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,P,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,M,N,40.0,10.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2,9,1,4,10,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cl_Berkeley_104,Berkeley_104,0.87085,63.59506,NaN,NaN,4.0,0000+633,117.63,1.22,C0000+633,3,104,0.52,3.3,4,282.0,419.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2,P,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,P,-,15.0,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,3,2,1044,5,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cl_Blanco_1,Blanco_1,1.06343,-29.92162,190.0,7.70,89.0,0001-302,14.97,-79.26,C0001-302,24,1,0.51,3.3,5,43.0,49.0,170.0,374.0,0.03,322.0,0.09,64,NaN,NaN,3,2,M,NaN,8.0,10105.0,NaN,NaN,NaN,NaN,20.0,1.0,1.0,1.0,0.00,380.0,-0.15,380.0,4.0,3.0,M,-,30.0,70.0,1,0.0,5.0,2.0,1.0,73.0,NaN,NaN,NaN,0,0,4,869,0,6,827,0,240.0,4.0,-0.15,70.0,3.0,0.02,NaN,3.0,138.0,123.0,NaN,NaN,NaN,NaN
3,C_0001+557,Stock_19,1.09607,56.02838,NaN,NaN,3.0,0001+557,116.35,-6.24,C0001+557,11,19,0.52,3.3,6,274.0,437.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,P,NaN,8.0,10105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,P,-,6.0,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,5,1051,2,7,1052,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_0005+611,Czernik_1,1.92972,61.41163,NaN,NaN,9.0,0005+611,117.73,-1.02,C0005+611,4,1,0.52,3.3,7,283.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,DO,19.0,4,2,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,P,-,12.0,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,6,4,0,8,8,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cl_Berkeley_1,Berkeley_1,2.40822,60.42822,NaN,NaN,5.0,0007+601,117.79,-2.03,C0007+601,3,1,0.53,3.3,8,284.0,419.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,P,-,10.0,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,7,7,0,9,9,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C_0007+609,King_13,2.53490,61.21153,NaN,NaN,7.0,0007+609,117.98,-1.28,C0007+609,10,13,0.53,3.3,9,285.0,329.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2,P,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,M,-,30.0,5.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,8,8,1050,10,3,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cl_Berkeley_60,Berkeley_60,4.42607,60.96103,NaN,NaN,4

In [31]:
#mwsc + webda
mwsc_webda_df = mwsc_df.join(webda_df.set_index('Simbad_ID'),
                            on='Simbad_ID',how='outer', lsuffix='_mwsc', rsuffix='_webda')
      
# + piskunov
mwsc_webda_piskunov_df = mwsc_webda_df.join(piskunov_df.set_index('Simbad_ID'), 
                            on='Simbad_ID', how='outer', rsuffix='_piskunov')

# + kharchenko
mwsc_webda_piskunov_kharchenko_df = mwsc_webda_piskunov_df.join(kharchenko_df.set_index('Simbad_ID'), 
                            on='Simbad_ID', how='outer', rsuffix='_kharchenko')

# + Salaris
mwsc_webda_piskunov_kharchenko_salaris_df = mwsc_webda_piskunov_kharchenko_df.join(salaris_df.set_index('Simbad_ID'), 
                            on='Simbad_ID', how='outer', rsuffix='_salaris')

# + vandenberg
mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_df = mwsc_webda_piskunov_kharchenko_salaris_df.join(
                            vandenbergh_df.set_index('Simbad_ID'), 
                            on='Simbad_ID', how='outer', rsuffix='_vandenbergh')


# + Gaia
mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_df = mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_df.join(
                            gaiaDR2_df.set_index('Simbad_ID'), 
                            on='Simbad_ID', how='outer', rsuffix='_cantat-gaudin')

# + Lynga
mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_lynga_df = mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_df.join(
                            lynga_df.set_index('Simbad_ID'), 
                            on='Simbad_ID', how='outer', rsuffix='_lynga')

#rename
OCs = mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_lynga_df.copy()

print(len(OCs.loc[pd.isnull(OCs['Simbad_ID'])]))

#reindex
OCs = OCs.reset_index(drop=True)

#if there are NaN rows; drop them
idx = OCs.index[pd.isnull(OCs['Simbad_ID'])]
OCs.drop(idx, inplace=True)

#reindex
OCs = OCs.reset_index(drop=True)


print(len(mwsc_df), len(webda_df), len(piskunov_df), len(kharchenko_df), len(salaris_df), len(vandenbergh_df),
     len(gaiaDR2_df), len(lynga_df), len(mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_df),
      len(mwsc_webda_piskunov_kharchenko_salaris_vandenbergh_gaia_lynga_df), len(OCs))

print(OCs.columns.values)

#dump to a file
OCs.to_csv('OCcompiled.csv', index=False)


0
2908 936 650 3005 70 595 1228 1078 3317 3356 3356
['Simbad_ID' 'Name_mwsc' 'broad_type_mwsc' 'cluster_status_mwsc' 'ra_mwsc'
 'dec_mwsc' 'lii_mwsc' 'bii_mwsc' 'core_radius_mwsc' 'central_radius_mwsc'
 'cluster_radius_mwsc' 'pm_ra_mwsc' 'pm_dec_mwsc' 'pm_tot_error_mwsc'
 'rad_vel_mwsc' 'rad_vel_error_mwsc' 'num_rad_vel_stars_mwsc'
 'num_core_stars_mwsc' 'num_central_stars_mwsc' 'num_cluster_stars_mwsc'
 'distance_mwsc' 'e_bv_mwsc' 'distance_modulus_mwsc' 'e_jk_mwsc'
 'e_jh_mwsc' 'delta_h_mwsc' 'log_age_mwsc' 'log_age_error_mwsc'
 'num_log_age_stars_mwsc' 'king_core_radius_mwsc'
 'king_core_radius_error_mwsc' 'king_tidal_radius_mwsc'
 'king_tidal_radius_error_mwsc' 'king_norm_factor_mwsc'
 'king_norm_factor_error_mwsc' 'reference_code_mwsc' 'cluster_type_mwsc'
 'metallicity_mwsc' 'metallicity_error_mwsc' 'num_metallicity_stars_mwsc'
 'comments_mwsc' 'class_mwsc' 'Name_webda' 'RA_2000_webda'
 'Dec_2000_webda' 'l_webda' 'b_webda' 'Dist_webda' 'Mod_webda'
 'EB-V_webda' 'Age_webda' 'ST_web

In [32]:
#a quick check to make sure that items matched up
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#print(OCs.loc[OCs['Simbad_ID'] == 'C_1440+697'].iloc[0])
print(OCs.loc[OCs['Simbad_ID'] == 'NGC_2682'].iloc[0])

Simbad_ID                                                                NGC_2682
Name_mwsc                                                                NGC_2682
broad_type_mwsc                                                                  
cluster_status_mwsc                                                              
ra_mwsc                                                                  08 51 23
dec_mwsc                                                                 +11 48.9
lii_mwsc                                                                  215.691
bii_mwsc                                                                   31.923
core_radius_mwsc                                                              0.1
central_radius_mwsc                                                          0.55
cluster_radius_mwsc                                                          1.03
pm_ra_mwsc                                                                  -7.31
pm_dec_mwsc     

In [33]:
#check for duplicates (by definition there should be none)
for index, row in  OCs.iterrows():
    check = OCs.loc[OCs['Simbad_ID'] == row['Simbad_ID']]
    if (len(check['Simbad_ID']) != 1):
        print('OCS', row['Simbad_ID'])
        print('Check', check['Simbad_ID'].values, len(check['Simbad_ID']))
        print('')

In [34]:
check = OCs.loc[OCs['Simbad_ID'].str.contains('Berkeley_69')]
print(len(check['Simbad_ID']))


1


In [35]:
check = OCs.loc[OCs['Simbad_ID'] == 'NAME_Trapezium_Cluster']
#check = mwsc_df.loc[mwsc_df['Simbad_ID'] == '[BDS2003]_73']
#check = mwsc_df.loc[mwsc_df['Name']== 'BDSB_73']
#print(len(check['Simbad_ID']))
check

,Simbad_ID,Name_mwsc,broad_type_mwsc,cluster_status_mwsc,ra_mwsc,dec_mwsc,lii_mwsc,bii_mwsc,core_radius_mwsc,central_radius_mwsc,cluster_radius_mwsc,pm_ra_mwsc,pm_dec_mwsc,pm_tot_error_mwsc,rad_vel_mwsc,rad_vel_error_mwsc,num_rad_vel_stars_mwsc,num_core_stars_mwsc,num_central_stars_mwsc,num_cluster_stars_mwsc,distance_mwsc,e_bv_mwsc,distance_modulus_mwsc,e_jk_mwsc,e_jh_mwsc,delta_h_mwsc,log_age_mwsc,log_age_error_mwsc,num_log_age_stars_mwsc,king_core_radius_mwsc,king_core_radius_error_mwsc,king_tidal_radius_mwsc,king_tidal_radius_error_mwsc,king_norm_factor_mwsc,king_norm_factor_error_mwsc,reference_code_mwsc,cluster_type_mwsc,metallicity_mwsc,metallicity_error_mwsc,num_metallicity_stars_mwsc,comments_mwsc,class_mwsc,Name_webda,RA_2000_webda,Dec_2000_webda,l_webda,b_webda,Dist_webda,Mod_webda,EB-V_webda,Age_webda,ST_webda,Z_webda,Diam_webda,Fe/H_webda,MRV_webda,pm RA_webda,pm Dec_webda,Measures_webda,Stars_webda,COCD_piskunov,Name_piskunov,GLON[deg]_piskunov,GLAT[deg]_piskunov,DistMod_piskunov,E(B-V)_piskunov,Dist[pc]_piskunov,logt[yr]_piskunov,rt[pc]_piskunov,e_rt[pc]_piskunov,logM[MSun]_piskunov,e_logM[MSun]_piskunov,rtA[pc]_piskunov,e_rtA[pc]_piskunov,logMA[MSun]_piskunov,e_logMA[MSun]_piskunov,MWSC_kharchenko,Name_kharchenko,Type_kharchenko,n_Type_kharchenko,RA[hr]_kharchenko,Dec[deg]_kharchenko,GLON[deg]_kharchenko,GLAT[deg]_kharchenko,r0[deg]_kharchenko,r1[deg]_kharchenko,r2[deg]_kharchenko,pmRA[mas/yr]_kharchenko,pmDec[mas/yr]_kharchenko,e_pm[mas/yr]_kharchenko,RV[km/s]_kharchenko,e_RV[km/s]_kharchenko,n_RV[km/s]_kharchenko,N1sr0_kharchenko,N1sr1_kharchenko,N1sr2_kharchenko,d[pc]_kharchenko,E(B-V)_kharchenko,appDistMod[mag]_kharchenko,E(J-Ks)_kharchenko,E(J-H)_kharchenko,dH_kharchenko,logt[yr]_kharchenko,e_logt[yr]_kharchenko,Nt_kharchenko,rc[pc]_kharchenko,e_rc[pc]_kharchenko,rt[pc]_kharchenko,e_rt[pc]_kharchenko,k[pc-2]_kharchenko,e_k[pc-2]_kharchenko,Src_kharchenko,SType_kharchenko,[Fe/H][Sun]_kharchenko,e_[Fe/H][Sun]_kharchenko,n_[Fe/H]_kharchenko,Name_salaris,dV_salaris,err_dV_salaris,[Fe/H]_salaris,err_[Fe/H]_salaris,t[Gyr]_salaris,err_t_salaris,Rgc[kpc]_salaris,z[pc]_salaris,flag_salaris,tJP94[Gyr]_salaris,Seq_vandenbergh,Name_vandenbergh,l[deg]_vandenbergh,Diam[pc]_vandenbergh,R[pc]_vandenbergh,Z[pc]_vandenbergh,E(B-V)_vandenbergh,logT[yr]_vandenbergh,SimbadName_vandenbergh,_RA[deg]_vandenbergh,_Dec[deg]_vandenbergh,_RAJ2000_cantat-gaudin,_DEJ2000_cantat-gaudin,Name_cantat-gaudin,RAJ2000[deg]_cantat-gaudin,DEJ2000[deg]_cantat-gaudin,GLON[deg]_cantat-gaudin,GLAT[deg]_cantat-gaudin,r50[deg]_cantat-gaudin,Nstars_cantat-gaudin,pmRA[mas/yr]_cantat-gaudin,pmDE[mas/yr]_cantat-gaudin,plx[mas]_cantat-gaudin,dmode[pc]_cantat-gaudin,Rgc[pc]_cantat-gaudin,SimbadName_cantat-gaudin,Name_lynga,ra_lynga,dec_lynga,distance_lynga,log_age_lynga,angular_diameter_lynga,alt_name_lynga,lii_lynga,bii_lynga,iau_num_lynga,seq_code_lynga,seq_num_lynga,prec_ra_lynga,prec_dec_lynga,lund_record_num_lynga,ocl_num_lynga,ref_angular_diameter_lynga,ref_distance_lynga,ref_log_age_lynga,metallicity_lynga,ref_metallicity_lynga,e_bv_lynga,ref_e_bv_lynga,type_flag_lynga,ref_type_flag_lynga,tr_concent_class_lynga,tr_range_class_lynga,tr_richness_class_lynga,tr_nebulosity_lynga,sb_bs_mag_lynga,sb_spect_code_lynga,sb_total_mag_lynga,sk_total_mag_lynga,sk_bv_color_lynga,sk_num_stars_lynga,ja_star_num_lynga,ja_class_lynga,ja_max_class_lynga,ja_richness_lynga,ja_e_bv_lynga,ref_ja_e_bv_lynga,ja_bv_turnoff_lynga,ref_ja_bv_turnoff_lynga,ly_tr_concent_class_lynga,ly_tr_range_class_lynga,ly_tr_richness_class_lynga,ly_tr_nebulosity_lynga,ly_member_stars_lynga,ly_angular_diameter_lynga,ly_refs_flag_lynga,radvel_weight_lynga,radvel_lynga,radvel_weight_class_lynga,ref_radvel1_lynga,ref_radvel2_lynga,ref_radvel3_lynga,ref_radvel4_lynga,ref_radvel5_lynga,basel_spect_code_lynga,basel_color_type_lynga,neg_ra_tracer_lynga,neg_lii_tracer_lynga,neg_seq_num_tracer_lynga,pos_ra_tracer_lynga,pos_lii_tracer_lynga,pos_seq_num_tracer_lynga,jdl_distance_lynga,jdl_distance_weight

In [36]:
check = OCs.loc[pd.isna(OCs['Simbad_ID'])]
print(len(check['Simbad_ID']))
#check

0


## Check for duplicates in RA and Dec

In [37]:
def getCoord(row):

    if (pd.notna(row['_RAJ2000_cantat-gaudin']) and pd.notna(row['_DEJ2000_cantat-gaudin'])):
        return SkyCoord(ra=row['_RAJ2000_cantat-gaudin']+' hours', dec=row['_DEJ2000_cantat-gaudin']+' degree', frame='icrs')
    
    elif (pd.notna(row['ra_mwsc']) and pd.notna(row['dec_mwsc'])):
        return SkyCoord(ra=row['ra_mwsc']+' hours', dec=row['dec_mwsc']+' degree', frame='icrs')
        
    elif (pd.notna(row['RA[hr]_kharchenko']) and pd.notna(row['Dec[deg]_kharchenko'])):
        return SkyCoord(ra=row['RA[hr]_kharchenko']*units.hourangle, dec=row['Dec[deg]_kharchenko']*units.degree, frame='icrs')

    elif (pd.notna(row['GLON[deg]_piskunov']) and pd.notna(row['GLAT[deg]_piskunov'])):
        return SkyCoord(l=row['GLON[deg]_piskunov']*units.degree, b=row['GLAT[deg]_piskunov']*units.degree, frame='galactic').icrs

    elif (pd.notna(row['ra_lynga']) and pd.notna(row['dec_lynga'])):
        return SkyCoord(ra=row['ra_lynga']*units.degree, dec=row['dec_lynga']*units.degree, frame='icrs')
    
    elif (pd.notna(row['_RA[deg]_vandenbergh']) and pd.notna(row['_Dec[deg]_vandenbergh'])):
        return SkyCoord(ra=row['_Dec[deg]_vandenbergh']*units.degree, dec=row['_Dec[deg]_vandenbergh']*units.degree, frame='icrs')

    elif (pd.notna(row['RA_2000_webda']) and pd.notna(row['Dec_2000_webda'])):
        return SkyCoord(ra=row['RA_2000_webda']+' hours', dec=row['Dec_2000_webda']+' degree', frame='icrs')
            
    elif (pd.notna(row['GLON[deg]_cantat-gaudin']) and pd.notna(row['GLAT[deg]_cantat-gaudin'])):
        return SkyCoord(l=row['GLON[deg]_cantat-gaudin']*units.degree, b=row['GLAT[deg]_cantat-gaudin']*units.degree, frame='galactic').icrs

    elif (pd.notna(row['l_webda']) and pd.notna(row['b_webda'])):
        return SkyCoord(l=row['l_webda']*units.degree, b=row['b_webda']*units.degree, frame='galactic').icrs
    
    else:
        print('NO RA, Dec : ', row['Simbad_ID'])
        return False

In [38]:
#first get the coordinates in lists
RA = []
Dec = []
for index, row in OCs.iterrows():
    #RA and Dec
    c = getCoord(row)
    if (c):
        RA.append(c.ra.degree)
        Dec.append(c.dec.degree)

catalog = SkyCoord(ra = RA*units.degree, dec = Dec*units.degree)

In [39]:
#now match to the full catalog to see if there are duplicates
#I think these are OK overlaps (really different clusters), 
#and now using differnt order in RA and Dec, I don't see most of them below anymore
#!!! OVERLAP NGC_3590 Hogg_12 [0.46110687] 253 2967
#!!! OVERLAP C_0925-549 Ruprecht_77 [0.90177733] 473 3360
#!!! OVERLAP NGC_6997 NGC_6996 [0.91921481] 3238 3250
#!!! OVERLAP Cl_Platais_8 NAME_HIP_67014_Cluster [0.44914403] 3260 3266
#!!! OVERLAP AH03_J0822-36.4 NGC_2579 [0.97738812] 888 3046 #not sure about this; NGC 2579 is labelled as HII region in Simbad
#!!! OVERLAP Cl_Pismis_24 NGC_6357 [0.97546562] 3134 3347 #not sure about this; NGC 6357 is labelled as HII region in Simbad

#should I change Trapezium in lynga to M42? I think yes
#!!! OVERLAP NAME_Trapezium_Cluster M_42 [0.88223261] 3325 2918

max_sep = 1.0 * units.arcmin
nover = 0
for index, row in OCs.iterrows():
    c = getCoord(row)
    idx, d2d, d3d = c.match_to_catalog_sky(catalog, nthneighbor=2) #first neighbor is itself
    #print(index, row['Name'], OCs.iloc[int(idx)]['Name'], d2d.degree)
    if (d2d < max_sep and idx != index):
        print('!!! OVERLAP',row['Simbad_ID'], OCs.iloc[int(idx)]['Simbad_ID'], d2d.arcminute, index, idx)
        nover += 1

print(nover)
        


!!! OVERLAP NGC_6997 NGC_6996 [0.91921481] 3213 3225
!!! OVERLAP NGC_6996 NGC_6997 [0.91921481] 3225 3213
2
